In [108]:
import warnings
warnings.filterwarnings('once')


In [109]:

import numpy as np
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.optimizers import Adam
import os
from glob import glob

In [110]:
IMAGE_SIZE = [224, 224]
TRAIN_PATH = 'Datasets/train'
VALID_PATH = 'Datasets/test'
BATCH_SIZE = 32
EPOCHS = 1
MODEL_PATH = 'chest_xray.h5'

In [111]:
def create_model():
    base_model = VGG16(input_shape=[224,224,3], weights='imagenet', include_top=False)
    for k in base_model.layers:
        k.trainable = False

    z = Flatten()(base_model.output)
    output_layer = Dense(len(glob(f'{TRAIN_PATH}/*')), activation='softmax')(z)

    model = Model(inputs=base_model.input, outputs=output_layer)
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [112]:
def get_data_generators():
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        rotation_range = 10,
        zoom_range=0.25,
        horizontal_flip=True,
        vertical_flip=False,

    )
    
    test_datagen = ImageDataGenerator(rescale=1./255)
    
    train_generator = train_datagen.flow_from_directory(
        TRAIN_PATH,
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical'
    )
    
    validation_generator = test_datagen.flow_from_directory(
        VALID_PATH,
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical'
    )
    
    return train_generator, validation_generator

In [113]:
def train_model(model, train_generator, validation_generator):
    
    model.fit(
        train_generator,
        epochs=EPOCHS,
        verbose=1,
        validation_data=validation_generator,
        steps_per_epoch=len(train_generator),
        validation_steps=len(validation_generator),

    )
    model.save(MODEL_PATH)


In [114]:
def predict_image(model, img_path):
    img = load_img(img_path, target_size=IMAGE_SIZE)
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    prediction = model.predict(x)
    return prediction

In [105]:
model = create_model()
train_gen, val_gen = get_data_generators()
train_model(model, train_gen, val_gen)
model = load_model(MODEL_PATH)
img_path = 'Datasets/val/NORMAL/NORMAL2-IM-1431-0001.jpeg'
prediction = predict_image(model, img_path)
result = np.argmax(prediction)
if result == 0:
    print("Person is Affected By PNEUMONIA")
else:
    print("Result is Normal")

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
163/163 [==============================] - 755s 5s/step - loss: 0.2589 - accuracy: 0.9091 - val_loss: 0.3000 - val_accuracy: 0.8894


1/1 [==============================] - 0s 200ms/step
Person is Affected By PNEUMONIA


In [106]:
!streamlit run streamlit_app.py

python3.11(54432) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8502
  Network URL: http://10.11.37.4:8502

1/1 [==============================] - 0s 276ms/step
^C
  Stopping...


hi
